In [1]:
import os
import re
import math
import logging
import random
import functools

import sqlite3
import json
import pandas as pd
import numpy as np
import itertools as tt

from typing import List, Tuple
#from collections import Counter
import matplotlib.pyplot as plt
Vector = List[float]
Matrix = List[List[float]]

logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', datefmt='%Y-%m-%d %I:%M:%S')
logging.getLogger().setLevel(logging.INFO)
logging.info("Logging set.")

2020-12-27 09:27:20 INFO: Logging set.


In [2]:
def get_integer_input(filename):
    retval = None
    with open(filename) as IN:
        retval = [int(x) for x in IN.read().splitlines()]
    logging.info("Got {} integers from {}.".format(len(retval),filename))
    return retval

def get_string_input(filename):
    retval = None
    with open(filename) as IN:
        retval = IN.read().splitlines()
    logging.info("Got {} lines from {}.".format(len(retval),filename))
    return retval

def reverse_graph(A):
    ''' Return the transpose of the adjency matrix A
            A is a dict of dicts;  A[x][y] = value
            retval[x][y] = A[y][x]
    '''
    retval = dict()
    for row in A.keys():
        retval[row] = dict()
    for row, cols in A.items():
        for col in cols.keys():
            retval[col][row] = A[row][col]
    return retval


def vector_scale(k, vec):
    return [k * v for v in vec]


def vector_sum(vec1, vec2):
    assert len(vec1) == len(vec2)
    return [v1 + v2 for v1, v2 in zip(vec1, vec2)]


def vector_equal(vec1, vec2):
    retval = False
    if len(vec1) == len(vec2):
        retval = True
        for v1, v2 in zip(vec1, vec2):
            if v1 != v2:
                retval = False
                break
    return retval


def l1_norm(vec):
    return sum([abs(v) for v in vec])

## Day 1:  Report Repair

### Problem 1: Find the two entries that sum to 2020; what do you get if you multiply them together?

### Problem 2: Find the three entries that sum to 2020; what do you get if you multiply them together?

In [3]:
entries = get_integer_input('prob1expenses.txt')

sols1 = [(x,y,x+y,x*y) for x in entries for y in entries if x < y and x + y == 2020]
sols2 = [(x,y,z,x+y+z,x*y*z) for x in entries for y in entries for z in entries if x < y < z and x + y + z == 2020]
for s in sols1:
    logging.info("Problem 1 solution:  x = {}, y = {}, sum = {}, product = {}".format(*s))
for s in sols2:
    logging.info("Problem 2 solution:  x = {}, y = {}, z = {}, sum = {}, product = {}".format(*s))


2020-12-27 09:27:20 INFO: Got 200 integers from prob1expenses.txt.
2020-12-27 09:27:20 INFO: Problem 1 solution:  x = 409, y = 1611, sum = 2020, product = 658899
2020-12-27 09:27:20 INFO: Problem 2 solution:  x = 250, y = 485, z = 1285, sum = 2020, product = 155806250


## Day 2:  Password Philosophy

### Problem 3: Find the number of "valid" passwords.

### Problem 4: Find the number of "valid" passwords with a different definition of valid.

In [4]:
def is_valid_q3(lo, hi, char, pswd):
    ''' Valid passwords for question 3 '''
    retval = False
    assert lo <= len(pswd) and hi <= len(pswd)
    char_count = pswd.count(char)
    if lo <= char_count and char_count <= hi:
        retval = True
    return retval

def is_valid_q4(pos1, pos2, char, pswd):
    ''' Valid passwords for question 4 '''
    retval = False
    assert pos1 <= len(pswd) and pos2 <= len(pswd)
    if pswd[pos1-1] != pswd[pos2-1]:
        if char in [ pswd[pos1-1], pswd[pos2-1] ]:
            retval = True
    return retval

##################################################################

lines = get_string_input('prob3passwords.txt')

valid_q3, valid_q4 = 0, 0

lineFormat = re.compile('^(\d+)-(\d+) ([a-z]): ([a-z]+)\s?$')
for line in lines:
    m1 = lineFormat.match(line)
    if m1 == None:
        logging.error(line)
    else:
        lo, hi, char, pswd = m1.groups(1)
        if is_valid_q3(int(lo), int(hi), char, pswd):
            valid_q3 += 1
        if is_valid_q4(int(lo), int(hi), char, pswd):
            valid_q4 += 1

logging.info("Problem 3: {} of {} are valid.".format(valid_q3, len(lines) ))
logging.info("Problem 4: {} of {} are valid.".format(valid_q4, len(lines) ))        

2020-12-27 09:27:20 INFO: Got 1000 lines from prob3passwords.txt.
2020-12-27 09:27:20 INFO: Problem 3: 614 of 1000 are valid.
2020-12-27 09:27:20 INFO: Problem 4: 354 of 1000 are valid.


## Day 3:  Toboggan Trajectory

### Problem 5: Starting at the top-left corner of your map and following a slope of right 3 and down 1, how many trees would you encounter?

### Problem 6: What do you get if you multiply together the number of trees encountered on each of the listed slopes?

In [5]:
lines = get_string_input('prob5trees.txt')

line_len = len(lines[0])
tree_counts = []

# Problem 5 is computed in the second iteration of the following loop in part 6.
for step_col, step_row in [[1,1], [3,1], [5,1], [7,1], [1,2]]:
    row, col, tree_count = 0, 0, 0
    while row < len(lines)-1:
        row += step_row
        col = (col + step_col) % line_len
        if lines[row][col] == '#':
            tree_count += 1
    tree_counts.append(tree_count)

product = functools.reduce(lambda x, y: x * y, tree_counts, 1)
logging.info("Problem 5: for slope [3,1], tree_count = {}".format(tree_counts[1],))
logging.info("Problem 6: tree_counts = {}, Product = {}".format(tree_counts, product))

2020-12-27 09:27:20 INFO: Got 323 lines from prob5trees.txt.
2020-12-27 09:27:20 INFO: Problem 5: for slope [3,1], tree_count = 156
2020-12-27 09:27:20 INFO: Problem 6: tree_counts = [79, 156, 85, 82, 41], Product = 3521829480


## Day 4:  Passport Processing

### Problem 7: How many passports are valid?

### Problem 8: How many passports are valid with stricter guidelines?

In [6]:
def process_passports(lines):
    ''' Process the input into a list of dictionarys, each dictionary a passport. '''
    retval = []
    passport = dict()
    for line in lines:
        if line == '':  # indicates break between passports
            retval.append(passport)
            passport = dict()
        else:
            for pair in line.split():
                key, val = pair.split(':')
                passport[key] = val
    retval.append(passport) # add final passport
    return retval


def is_valid_q7(p):
    ''' Valid passport for question 7. 
            p is a dictionary with 8 possible keys
    '''
    retval = True
    for key in ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']:
        if p.get(key, None) == None:
            retval = False
    return retval


def is_valid_q8(p):
    ''' Valid passport for question 7. 
            p is a dictionary with 8 possible keys
    '''    
    retval = True

    # byr
    val = int(p.get('byr','1900'))
    if val < 1920 or val > 2002:
        return False
    
    # iyr
    val = int(p.get('iyr','1900'))
    if val < 2010 or val > 2020:
        return False

    # eyr
    val = int(p.get('eyr','1900'))
    if val < 2020 or val > 2030:
        return False

    # hgt
    val = p.get('hgt','0cm')
    m_cm = re.match('^(\d+)cm$', val)
    m_in = re.match('^(\d+)in$', val)
    if m_cm == None and m_in == None:
        return False
    elif m_cm != None:
        cm_val = int(m_cm.groups(1)[0])
        if cm_val < 150 or cm_val > 193:
            return False
    else:
        in_val = int(m_in.groups(1)[0])
        if in_val < 59 or in_val > 76:
            return False

    # hcl
    if re.match('^#[0-9a-f]{6}$', p.get('hcl','')) == None:
        return False
    
    # ecl
    if p.get('ecl','') not in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']:
        return False

    # pid
    if re.match('^[0-9]{9}$', p.get('pid','')) == None:
        return False

    return retval

########################################################################

lines = get_string_input('prob7passports.txt')
passports = process_passports(lines)

valid_q7, valid_q8 = 0, 0
for p in passports:
    if is_valid_q7(p):
        valid_q7 += 1
    if is_valid_q8(p):
        valid_q8 += 1

logging.info("Problem 7: {} passports out of {} are valid.".format(valid_q7, len(passports)))
logging.info("Problem 8: {} passports out of {} are valid.".format(valid_q8, len(passports)))
        

2020-12-27 09:27:20 INFO: Got 1136 lines from prob7passports.txt.
2020-12-27 09:27:20 INFO: Problem 7: 237 passports out of 290 are valid.
2020-12-27 09:27:20 INFO: Problem 8: 172 passports out of 290 are valid.


## Day 5:  Binary Boarding

### Problem 9: What is the highest seat id? 

### Problem 10:  What is your seat id (the one missing in the middle)?

In [7]:
def get_row(r):
    return sum([2**(6-i) for i, ch in enumerate(r) if ch == 'B'])
    
def get_seat(s):
    return sum([2**(2-i) for i, ch in enumerate(s) if ch == 'R'])
    
def get_seat_id(row, seat):
    return 8*row + seat

#####################################################################

seats = get_string_input('prob9seats.txt')

sids = []
for seat in seats:
    m1 = re.match('^([FB]{7})([RL]{3})$', seat)
    if m1 == None:
        logging.error("Could not parse: {}".format(seat,))
    else:
        r, s = m1.groups(1)
        sids.append(get_seat_id(get_row(r), get_seat(s)))

logging.info("Problem 9: Max seat id = {}.".format(max(sids),))
for sid in sids:
    if sid+1 not in sids and sid+2 in sids:
        logging.info("Problem 10: {} and {} are taken but {} is not.".format(sid, sid+2, sid+1))

2020-12-27 09:27:20 INFO: Got 867 lines from prob9seats.txt.
2020-12-27 09:27:20 INFO: Problem 9: Max seat id = 880.
2020-12-27 09:27:20 INFO: Problem 10: 730 and 732 are taken but 731 is not.


## Day 6:  Custom Customs

### Problem 11: For each group, count the number of questions to which anyone answered "yes". What is the sum of those counts?

### Problem 12:  For each group, count the number of questions to which everyone answered "yes". What is the sum of those counts?

In [8]:
def process_surveys(lines):
    ''' Process the input into a list of lists, each sublist is a survey. '''
    retval = []
    group = []
    for line in lines:
        if line == '':  # indicates a break between groups
            retval.append(group)
            group = []
        else:
            group.append(line)
    retval.append(group) # append last group
    return retval

###############################################################

lines = get_string_input('prob11surveys.txt')

groups = process_surveys(lines)
logging.info("There are {} groups.".format(len(groups),))

count_q11 = 0
count_q12 = 0
for group in groups:
    yeses_q11 = set()
    yeses_q12 = None
    for g in group:
        yeses_q11 = yeses_q11.union(set(g))
        if yeses_q12 == None:  # Intialize the intersection.
            yeses_q12 = set(g)
        else:
            yeses_q12 = yeses_q12.intersection(set(g))
    count_q11 += len(yeses_q11)
    count_q12 += len(yeses_q12)
        
logging.info("Problem 11: The count is {}.".format(count_q11,))
logging.info("Problem 12: The count is {}.".format(count_q12,))

2020-12-27 09:27:20 INFO: Got 2132 lines from prob11surveys.txt.
2020-12-27 09:27:20 INFO: There are 478 groups.
2020-12-27 09:27:20 INFO: Problem 11: The count is 6521.
2020-12-27 09:27:20 INFO: Problem 12: The count is 3305.


## Day 7: Handy Haversacks

### Problem 13: How many bag colors can eventually contain at least one shiny gold bag?

### Problem 14: How many individual bags are required inside your single shiny gold bag?

In [9]:
def process_rules(rules):
    ''' Process the input into a dict of dicts:
            retval[color1][color2] = count
            # bag color1 contains exactly count bags of color2
    '''
    retval = dict()
    for rule in rules:
        m1 = re.match('^(.*) bags contain (.*)\.$', rule)
        if m1 == None:
            logging.error("Could not match outer rule: '{}'".format(rule,))
        else:
            outer, contents = m1.groups(1) # [color of outer bag, contents of outer bag]
            retval[outer] = dict()

            for b in contents.split(', '):
                m2 = re.match('^(\d)+ (.*) bag[s]?$', b)  # 
                if m2 != None:
                    num, inner = m2.groups(1)
                    retval[outer][inner] = int(num)
                elif re.match('^no other bags$', b) == None:
                    logging.error("could not match '{}'  '{}'".format(outer, b))
    return retval


def is_descendent(myDict, ancestor, spawn):
    ''' Recursive boolean to test if spawn is a descendent of ancestor. '''
    retval = False
    if ancestor == spawn:  # Don't ask how this is possible.
        retval = True
    else:
        for child in myDict[ancestor].keys():
            if is_descendent(myDict, child, spawn) == True:
                retval = True
                break
    return retval


def count_descendents(myDict, ancestor):
    ''' Recursive count of descendent of ancestor. '''
    retval = 0
    for child, value in myDict[ancestor].items():
        retval += myDict[ancestor][child] * (1 + count_descendents(myDict, child))
    return retval

###################################################################################

rules = get_string_input('prob13bags.txt')
rules_dict = process_rules(rules)  # rules_dict[outer][inner] = count

myColor = 'shiny gold'
ancestors = [c for c in rules_dict.keys() if is_descendent(rules_dict, c, myColor) and c != myColor]
logging.info("Problem 13: The number of bags that contain '{}'' is {}.".format(myColor, len(ancestors)))

prodigy = count_descendents(rules_dict, myColor)
logging.info("Problem 14: The number of bags contained in '{}' is {}.".format(myColor, prodigy))


2020-12-27 09:27:20 INFO: Got 594 lines from prob13bags.txt.
2020-12-27 09:27:20 INFO: Problem 13: The number of bags that contain 'shiny gold'' is 296.
2020-12-27 09:27:20 INFO: Problem 14: The number of bags contained in 'shiny gold' is 9339.


## Day 8:  Handheld Halting

### Problem 15: Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

### Problem 16: Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [10]:
def process_commands(lines):
    retval = []
    for line in lines:
        m1 = re.match('^([a-z]{3}) ([\-\+]{1})([\d]+)$', line)
        if m1 == None:
            logging.error("Invalid command line: {}".format(line,))
        else:
            command, sign, num = m1.groups(1)
            num = int(num)
            if sign == '-':
                num = -1*num
            retval.append({'c':command, 'n':num})
    return retval


def run_command(com, accum, index):
    if com['c'] == 'acc':
        accum += com['n']
    elif com['c'] == 'jmp':
        index += com['n'] - 1  # will be added on return
    return accum, index+1


def modify_program(commands, index):
    ''' Return a boolean indicating if swap made at index. '''
    retval = False
    if commands[index]['c'] == 'jmp':
        commands[index]['c'] = 'nop'
        retval = True
    elif commands[index]['c'] == 'nop':
        commands[index]['c'] = 'jmp'
        retval = True
    return retval
    

def run_program(commands):
    ''' Run program until repeat of index or program terminates. 
            Return boolean, index, accumulator
    '''
    completed = False
    accum, index = 0, 0
    used = set()
    while index not in used:
        used.add(index)
        accum, index = run_command(commands[index], accum, index)
        if index == len(commands):
            completed = True
            break
    return completed, index, accum
    
    
###########################################################################

lines = get_string_input('prob15code.txt')

commands = process_commands(lines)

completed, index, accum = run_program(commands)
logging.info("Problem 15: Before index {} is repeated the accumulator is {}.".format(index, accum))

for chng in range(len(commands)):
    if modify_program(commands, chng):
        completed, index, accum = run_program(commands)
        modify_program(commands, chng)  # change back
        if completed == True:
            break

logging.info("Problem 16: Changed index {} and program terminated with accumulator {}.".format(chng, accum))
    
        

2020-12-27 09:27:20 INFO: Got 675 lines from prob15code.txt.
2020-12-27 09:27:20 INFO: Problem 15: Before index 641 is repeated the accumulator is 1928.
2020-12-27 09:27:20 INFO: Problem 16: Changed index 407 and program terminated with accumulator 1319.


## Day 9:  Encoding Error

### Problem 17: The first step of attacking the weakness in the XMAS data is to find the first number in the list (after the preamble) which is not the sum of two of the 25 numbers before it. What is the first number that does not have this property?

### Problem 18: What is the encryption weakness in your XMAS-encrypted list of numbers?

In [11]:
def is_valid(numbers, index):
    retval = False
    for i in range(index-25, index):
        for j in range(i+1, index):
            if numbers[i] + numbers[j] == numbers[index]:
                retval = True
                break
        if retval == True:
            break
    return retval

################################################################

numbers = get_integer_input('prob17numbers.txt')

for index in range(25,len(numbers)):
    if not is_valid(numbers, index):
        break
logging.info("Problem 17: At index {} the value {} is not valid".format(index, numbers[index]))

target = numbers[index]
myRange = None
for start in range(len(numbers)):
    for end in range(start+1, len(numbers)):
        mySum = sum(numbers[start:end+1])
        if mySum == target:
            myRange = [start, end]
        elif mySum > target:
            break
    if myRange != None:
        break

weakness = min(numbers[start:end+1]) + max(numbers[start:end+1])
logging.info("Problem 18: The range is {} and the weakness is {}".format(myRange, weakness))

        

2020-12-27 09:27:20 INFO: Got 1000 integers from prob17numbers.txt.
2020-12-27 09:27:20 INFO: Problem 17: At index 640 the value 1309761972 is not valid
2020-12-27 09:27:21 INFO: Problem 18: The range is [524, 540] and the weakness is 177989832


## Day 10: Adapter Array

### Problem 19: Find a chain that uses all of your adapters to connect the charging outlet to your device's built-in adapter and count the joltage differences between the charging outlet, the adapters, and your device. What is the number of 1-jolt differences multiplied by the number of 3-jolt differences?

### Problem 20: What is the total number of distinct ways you can arrange the adapters to connect the charging outlet to your device?

In [12]:
numbers = sorted(get_integer_input('prob19numbers.txt'))

diffs = [0,0,0,1]  # The last diff (from the highest value in the list) is 3, including here
for i in range(len(numbers)):
    diff = numbers[i] if i == 0 else numbers[i] - numbers[i-1]
    #diff = numbers[i]
    #if i > 0:
    #    diff -= numbers[i-1]
    if diff > 3:
        logging.error("Distance too great:  {} {}".format(i-1, i))
    else:
        diffs[diff] += 1
logging.info("Problem 19: The difference counts are {}, the product is {}.".format(diffs, diffs[1] * diffs[3]))


paths = [0] * len(numbers)
for i in range(len(numbers)):
    if numbers[i] <= 3:
        paths[i] += 1
    for j in range(max(0,i-3), i):
        if numbers[i] - numbers[j] <= 3:
            paths[i] += paths[j]
logging.info("Problem 20: The number of paths to maximum joltage is {}.".format(paths[-1],))


2020-12-27 09:27:21 INFO: Got 107 integers from prob19numbers.txt.
2020-12-27 09:27:21 INFO: Problem 19: The difference counts are [0, 75, 0, 33], the product is 2475.
2020-12-27 09:27:21 INFO: Problem 20: The number of paths to maximum joltage is 442136281481216.


## Day 11: Seating System

### Problem 21: Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

### Problem 22: Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [13]:
def process_seats(lines):
    return [ [ch for ch in line] for line in lines]

def count_adjacent(layout, row, col):
    retval = 0
    m, n = len(layout), len(layout[0])    # rows, cols
    for r in range(max(0,row-1), min(row+2, m)):
        for c in range(max(0,col-1), min(col+2, n)):
            if (row != r or col != c) and layout[r][c] == '#':
                retval += 1
    return retval

def count_viewable(grid, row, col):
    retval = 0
    m, n = len(grid), len(grid[0])    # num rows, num cols
    #step         e,     ne,     n,      nw,      w,      sw,     s,     se
    for step in [[0,1], [-1,1], [-1,0], [-1,-1], [0,-1], [1,-1], [1,0], [1,1]]:
        r, c = row, col
        for k in range(max(m, n)):
            r, c = vector_sum([r, c], step)
            if r < 0 or r >= m or c < 0 or c >=n:
                break  # looked beyond edge
            elif grid[r][c] == 'L':
                break
            elif grid[r][c] == '#':
                retval += 1
                break
    return retval

def step_val(grid, row, col, version):
    assert version in ['adj', 'view']
    retval = grid[row][col]
    count, limit = None,  None
    if version == 'adj':
        count, limit = count_adjacent(grid, row, col), 4
    else:
        count, limit = count_viewable(grid, row, col), 5

    if retval == 'L' and count == 0:
        retval = '#'
    elif retval == '#' and count >= limit:
        retval = 'L'
    return retval

def step_grid(grid, version):
    retval = []
    m, n, changed = len(grid), len(grid[0]), False
    for r in range(m):
        retval.append( [step_val(grid, r, c, version) for c in range(n)] )
        changed |= not vector_equal(retval[r], grid[r])
    return changed, retval

def get_occupied(grid):
    return sum([sum([1 for ch in row if ch == '#']) for row in grid])

##########################################################################    

#lines = ['L.LL.LL.LL','LLLLLLL.LL','L.L.L..L..','LLLL.LL.LL','L.LL.LL.LL',
#        'L.LLLLL.LL','..L.L.....','LLLLLLLLLL','L.LLLLLL.L','L.LLLLL.LL']

lines = get_string_input('prob21.txt')

grid = process_seats(lines)
steps, changed = 0, True
while changed == True:
    changed, grid = step_grid(grid, 'adj')
    steps += 1
logging.info("Problem 21: Occupied seats after {} iterations is {}".format(steps, get_occupied(grid),))


grid = process_seats(lines)
steps, changed = 0, True
while changed == True:
    changed, grid = step_grid(grid, 'view')
    steps += 1
logging.info("Problem 22: Occupied seats after {} iterations is {}".format(steps, get_occupied(grid),))



2020-12-27 09:27:21 INFO: Got 97 lines from prob21.txt.
2020-12-27 09:27:24 INFO: Problem 21: Occupied seats after 82 iterations is 2368
2020-12-27 09:27:36 INFO: Problem 22: Occupied seats after 85 iterations is 2124


## Day 12: Rain Risk

### Problem 23: Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

### Problem 24: Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

In [14]:
def process_moves(lines):
    retval = []
    for line in lines:
        m1 = re.match('^([NSEWLRF]{1})([\d]+)$', line)
        if m1 == None:
            logging.error("Bad input {}".format(line,))
        else:
            retval.append([m1.groups(1)[0], int(m1.groups(1)[1])])
    return retval

def move_vert(vec, val):
    vec[1] += val

def move_horz(vec, val):
    vec[0] += val

def move_forward(vec, val):
    direc = vec[2]
    if direc == 'N':
        move_vert(vec, val)
    elif direc == 'S':
        move_vert(vec, -val)
    elif direc == 'W':
        move_horz(vec, -val)
    elif direc == 'E':
        move_horz(vec, val)
    
def rotate(vec, val):
    ''' Rotate anti-clockwise '''
    new_i = ['E','N','W','S'].index(vec[2]) + [0,90,180,270].index(val % 360)
    vec[2] = ['E','N','W','S'][new_i % 4]

def move_toward_way_point(ship, way, val):
    ship[0] += (val * way[0])
    ship[1] += (val * way[1])

def rotate_way(vec, val):
    ''' Rotate anti-clockwise '''
    for i in range([0, 90, 180, 270].index(val % 360)):
        vec[0], vec[1] =  -vec[1],  vec[0]

####################################################################
        
lines = get_string_input('prob23.txt')
#lines = ['F10','N3','F7','R90','F11']

moves = process_moves(lines)

pos = [0, 0, 'E']  # x, y, direction
for mv in moves:
    val = mv[1]
    if mv[0] in ['R', 'S', 'W']:
        val *= -1
    if mv[0] in ['R', 'L']:
        rotate(pos, val)
    elif mv[0] in ['N', 'S']:
        move_vert(pos, val)
    elif mv[0] in ['E', 'W']:
        move_horz(pos, val)
    elif mv[0] == 'F':
        move_forward(pos, val)
logging.info("Problem 23:  Ship position {} has distance {}.".format(pos,abs(pos[0]) + abs(pos[1]) ))

pos = [0, 0, 'E']  # x, y, direction
way = [10, 1]     # x, y
for mv in moves:
    val = mv[1]
    if mv[0] in ['R', 'S', 'W']:
        val *= -1
    if mv[0] in ['R', 'L']:
        rotate_way(way, val)
    elif mv[0] in ['N', 'S']:
        move_vert(way, val)
    elif mv[0] in ['E', 'W']:
        move_horz(way, val)
    elif mv[0] == 'F':
        move_toward_way_point(pos, way, mv[1])
logging.info("Problem 24:  Ship position {} has distance {}.".format(pos,abs(pos[0]) + abs(pos[1]) ))
logging.info("              Way point {} relative to ship.".format(way,))


2020-12-27 09:27:36 INFO: Got 788 lines from prob23.txt.
2020-12-27 09:27:36 INFO: Problem 23:  Ship position [466, -1165, 'W'] has distance 1631.
2020-12-27 09:27:36 INFO: Problem 24:  Ship position [57377, -1229, 'E'] has distance 58606.
2020-12-27 09:27:36 INFO:               Way point [-92, 15] relative to ship.


## Day 13: Shuttle Search

### Problem 25: What is the ID of the earliest bus you can take to the airport multiplied by the number of minutes you'll need to wait for that bus?

### Problem 26: What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?

In [15]:
def process_buses(lines):
    earliest = lines[0]
    buses = [int(x) for x in lines[1].split(',') if x != 'x']
    return int(earliest), buses


def process_buses_with_xs(lines):
    return [int(x) if x != 'x' else 0 for x in lines[1].split(',')]


def ext_euclidean_alg(a, b):  
    ''' Return gcd(a,b) and ints x,y such that ax + by = gcd(a,b) '''
    if a == 0 :   
        return b, 0, 1
    gcd, x, y = ext_euclidean_alg(b%a, a)  
    return gcd, y - (b//a) * x, x    
  
def chinese_remainder_theorem(r1, m1, r2, m2):
    ''' Return x, m where:   
            m = lcm[m1,m2]
            x = r1 mod (m1) and    
            x = r2 mod (m2)
            0 <= x < m
    '''
    gcd, n1, n2 = ext_euclidean_alg(m1,m2)
    mult = (r1 - r2) // gcd
    m = (m1*m2) // gcd 
    x = (r2 + m2*n2*mult) % m
    return x, m

#lines = ['939','7,13,x,x,59,x,31,19']
lines = get_string_input('prob25.txt')

start, buses = process_buses(lines)
s = min(zip([-start % bus for bus in buses], buses))
logging.info("Problem 25: Bus {} has shortest wait: {}.  Prod = {}".format(s[1], s[0], s[1]*s[0]))

buses = process_buses_with_xs(lines)
r1, m1 = 0, buses[0]
for i, m2 in enumerate(buses):
    if i > 0 and m2 > 0:
        r1, m1 = chinese_remainder_theorem(r1, m1, -i % m2, m2)

logging.info("Problem 26: The earliest timestamp is {}".format(r1,))

2020-12-27 09:27:36 INFO: Got 2 lines from prob25.txt.
2020-12-27 09:27:36 INFO: Problem 25: Bus 17 has shortest wait: 7.  Prod = 119
2020-12-27 09:27:36 INFO: Problem 26: The earliest timestamp is 1106724616194525


## Day 14: Docking Data

### Problem 27: What is the sum of all values left in memory after it completes?

### Problem 28: What is the sum of all values left in memory after it completes?

In [16]:
def get_new_mask(line):
    retval = None
    m1 = re.match('^mask = ([X01]{36})$', line)
    if m1 == None:
        logging.error("Bad mask:  {}".format(line,))
    else:
        retval = list(m1.groups(1)[0])
    return retval

def get_mem_addr(line):
    retval = None
    m1 = re.match('^mem\[([\d]+)\] = (\d+)', line)
    if m1 == None:
        logging.error("Bad mem:  {}".format(line,))
    else:
        retval = [int(m1.groups(1)[0]), int(m1.groups(1)[1])]
    return retval

def get_bit_array(num):
    return [1 if x=='1' else 0 for x in "{:036b}".format(num,)]
    
def add_mask(bits, mask):
    assert len(bits) == len(mask)
    return [int(mask[i]) if mask[i] != 'X' else bits[i] for i in range(len(bits))]

def add_mask2(bits, mask):
    assert len(bits) == len(mask)
    retval = [str(b) for b in bits]
    for i, b in enumerate(mask):
        if b in ['1', 'X']:
            retval[i] = b
    return retval

def get_val_from_bit_list(bits):
    retval = 0
    for i, b in enumerate(bits):
        if b == 1:
            retval += 2**(len(bits)-1-i)
    return retval

def get_new_mems(bits):
    ''' Return a list of all possible integers obtained from bits where all
            X's are replaced with 0 or 1.
    '''
    retval = []
    numX, numbits = bits.count('X'), len(bits)
    for num in range(2**numX):
        new_mem_loc = bits[:]
        insert_bits = get_bit_array(num)
        added = 0
        for i, b in enumerate(new_mem_loc):
            if b == 'X':
                new_mem_loc[i] = str(insert_bits[numbits-1-added])
                added += 1
        retval.append(get_val_from_bit_list([int(x) for x in new_mem_loc]))
    return retval

###############################################################################

lines = get_string_input('prob27.txt')
#lines=['mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X','mem[8] = 11','mem[7] = 101','mem[8] = 0']

mask, loc, val, values = None, None, None, dict()
for line in lines:
    if re.match('^mask = ([X01]{36})$', line) != None:
        mask = get_new_mask(line)
    elif re.match('^mem\[([\d]+)\] = (\d+)', line) != None:
        loc, val = get_mem_addr(line)
        bits = add_mask(get_bit_array(val), mask)
        values[loc] = get_val_from_bit_list(bits)
        
logging.info('Problem 27: The sum is {}'.format(sum([v for v in values.values()]),))

# lines = ['mask = 000000000000000000000000000000X1001X','mem[42] = 100',
#            'mask = 00000000000000000000000000000000X0XX','mem[26] = 1']

mask, loc, val, values = None, None, None, dict()
for line in lines:
    if re.match('^mask = ([X01]{36})$', line) != None:
        mask = get_new_mask(line)
    elif re.match('^mem\[([\d]+)\] = (\d+)', line) != None:
        loc, val = get_mem_addr(line)
        bits = add_mask2(get_bit_array(loc), mask)
        for new_mem_loc in get_new_mems(bits):
            values[new_mem_loc] = val

logging.info('Problem 28: The sum is {}'.format(sum([v for v in values.values()]),))





2020-12-27 09:27:36 INFO: Got 569 lines from prob27.txt.
2020-12-27 09:27:36 INFO: Problem 27: The sum is 5055782549997
2020-12-27 09:27:38 INFO: Problem 28: The sum is 4795970362286


## Day 15:  Rambunctious Recitation

### Problem 29: Given your starting numbers, what will be the 2020th number spoken?

### Problem 30: Given your starting numbers, what will be the 30000000th number spoken?

In [17]:
#nums = [0,3,6]
nums = [12,20,0,6,1,17,7]

targets = [2020, 30000000]
last = dict()
for r, val in enumerate(nums[:-1]):
    last[val] = r+1

spoke = nums[-1]
for r in range(len(nums), max(targets)+1):
    if r == targets[0]:
        logging.info("Problem 29: In round = {} num {} is spoken.".format(r, spoke))
    elif r == targets[1]:
        logging.info("Problem 30: In round = {} num {} is spoken.".format(r, spoke))

    next_spoke = 0 if last.get(spoke, None) == None else r - last[spoke]
    last[spoke] = r
    spoke = next_spoke


2020-12-27 09:27:38 INFO: Problem 29: In round = 2020 num 866 is spoken.
2020-12-27 09:28:01 INFO: Problem 30: In round = 30000000 num 1437692 is spoken.


## Day 16:  Ticket Translation

### Problem 31: What is your ticket scanning error rate?

### Problem 32: Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

In [18]:
def get_class_def(line):
    retval = None
    m1 = re.match('^(.*): (\d+)-(\d+) or (\d+)-(\d+)$', line)
    if m1 == None:
        logging.error("Line is not a class definition:  '{}'".format(line,))
    else:
        retval = [m1.groups(1)[0], [int(x) for x in m1.groups(1)[1:3]], [int(x) for x in m1.groups(1)[3:5]]]
    return retval

def process_ticket_input(lines):
    classes, myTicket, tickets = dict(), None, []
    state = 'get_classes'
    for line in lines:
        if line == '':
            state = None
        elif state == 'get_classes':
            cls = get_class_def(line)
            if cls != None:
                classes[cls[0]] = [cls[1], cls[2]]
        elif re.match('^your ticket:$', line) != None:
            state = 'get_ticket'
        elif state == 'get_ticket':
            myTicket = [int(x) for x in line.split(',')]
        elif re.match('^nearby tickets:$', line) != None:
            state = 'get_nearby'
        elif state == 'get_nearby':
            tickets.append([int(x) for x in line.split(',')])
        
    return classes, myTicket, tickets


def is_value_in_class(val, ranges):
    return any([r[0] <= val and val <= r[1] for r in ranges])    

def is_valid_ticket_val(val, cls):
    return any([is_value_in_class(val, cls[cl]) for cl in cls])

def is_valid_col(col, ranges, tickets):
    return all([is_value_in_class(t[col], ranges) for t in tickets])


    
lines = get_string_input('prob31.txt')
#lines = ['class: 1-3 or 5-7','row: 6-11 or 33-44','seat: 13-40 or 45-50','',\
# 'your ticket:','7,1,14','','nearby tickets:','7,3,47','40,4,50','55,2,20','38,6,12']
#lines = ['class: 0-1 or 4-19','row: 0-5 or 8-19','seat: 0-13 or 16-19','',\
# 'your ticket:','11,12,13','','nearby tickets:','3,9,18','15,1,5','5,14,9']

cls, myTicket, tickets = process_ticket_input(lines)
n = len(cls)

# Count number of invalid values and create valid_tickets.
scanning_error_rate = 0
valids = []
for t in tickets:
    s_e_r = sum([val for val in t if not is_valid_ticket_val(val, cls)])
    scanning_error_rate += s_e_r
    if s_e_r == 0:
        valids.append(t)  # Save ticket for problem 32.
logging.info("Problem 31: Scanning error rate = {}".format(scanning_error_rate,))
        
# Get the possible columns that work for each class
options = dict()
for cl in cls.keys():
    options[cl] = set([col for col in range(n) if is_valid_col(col, cls[cl], valids)])

# Iteratively assign columns to classes if only 1 possible column exists.
#  Remove assigned column from all possible-values of remaining classes.
assigned = dict()
while len(options) > 0:
    for cl in sorted(options.keys(), key = lambda x: options[x]):
        if len(options[cl]) == 1:
            remove_val = options.pop(cl).pop()
            assigned[cl] = remove_val
            for cl2 in options:
                options[cl2].discard(remove_val)

result = 1
for cl, index in assigned.items():
    if re.match('^departure.*$', cl) != None:
        result *= myTicket[index]
logging.info('Problem 32: Product = {}'.format(result, ))

2020-12-27 09:28:01 INFO: Got 263 lines from prob31.txt.
2020-12-27 09:28:01 INFO: Problem 31: Scanning error rate = 22057
2020-12-27 09:28:01 INFO: Problem 32: Product = 1093427331937


## Day 17: Conway Cubes

### Problem 33:  How many cubes are left in the active state after the sixth cycle?

### Problem 34: Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?

In [19]:
def get_adj_active(L, center):
    retval = 0
    if len(center) == 3:
        for v in tt.product(np.arange(-1,2), np.arange(-1,2), np.arange(-1,2)):
            if l1_norm(v) > 0 and tuple(vector_sum(center,v)) in L:
                retval += 1
    elif len(center) == 4:
        for v in tt.product(np.arange(-1,2), np.arange(-1,2), np.arange(-1,2), np.arange(-1,2)):
            if l1_norm(v) > 0 and tuple(vector_sum(center,v)) in L:
                retval += 1
    return retval


def step(L, dim=3):
    retval = set()
    xs, ys, zs = [p[0] for p in L], [p[1] for p in L], [p[2] for p in L]
    minx, maxx = min(xs), max(xs)
    miny, maxy = min(ys), max(ys)
    minz, maxz = min(zs), max(zs)
        
    if dim == 3:
        for new_p in tt.product(np.arange(minx-1,maxx+2), 
                                np.arange(miny-1,maxy+2), 
                                np.arange(minz-1,maxz+2)):
            adj = get_adj_active(L, new_p)
            if new_p not in L and adj == 3:
                retval.add(new_p)
            elif new_p in L and adj in [2,3]:
                retval.add(new_p)

    if dim == 4:
        ws = [p[3] for p in L]
        minw, maxw = min(ws), max(ws)
        for new_p in tt.product(np.arange(minx-1,maxx+2), 
                                np.arange(miny-1,maxy+2), 
                                np.arange(minz-1,maxz+2),
                                np.arange(minw-1,maxw+2)):
            adj = get_adj_active(L, new_p)
            if new_p not in L and adj == 3:
                retval.add(new_p)
            elif new_p in L and adj in [2,3]:
                retval.add(new_p)
        
    return retval


lines = get_string_input('prob33.txt')
lattice = set()
for y, line in enumerate(reversed(lines)):
    for x, ch in enumerate(line):
        if ch == '#':
            lattice.add(tuple([x,y,0]))
        
for _ in range(6):
    lattice = step(lattice)
logging.info("Problem 33: Number of active after six rounds is {}.".format(len(lattice),))


lines = get_string_input('prob33.txt')
lattice = set()
for y, line in enumerate(reversed(lines)):
    for x, ch in enumerate(line):
        if ch == '#':
            lattice.add(tuple([x,y,0,0]))
        
for _ in range(6):
    lattice = step(lattice, dim=4)
logging.info("Problem 34: Number of active after six rounds is {}.".format(len(lattice),))


2020-12-27 09:28:01 INFO: Got 8 lines from prob33.txt.
2020-12-27 09:28:02 INFO: Problem 33: Number of active after six rounds is 267.
2020-12-27 09:28:02 INFO: Got 8 lines from prob33.txt.
2020-12-27 09:28:29 INFO: Problem 34: Number of active after six rounds is 1812.


## Day 18: Operation Order

### Problem 35: Evaluate the expression on each line of the homework; what is the sum of the resulting values?

### Problem 36: What do you get if you add up the results of evaluating the homework problems using these new rules?

In [20]:
def evaluate(s, pref = None):
    # look for parentheses and do those first
    while s.count(')') > 0: 
        cl_paren = s.index(')')      # Do first occurrence of ')'
        op_paren = s[:cl_paren].rindex('(')  # Matching '(' paren
        evald = evaluate(s[op_paren+1:cl_paren], pref)
        s = "{}{}{}".format(s[:op_paren], evald, s[cl_paren+1:])
    
    # s should have no parentheses now
    if pref != None:
        while s.count(pref) > 0:
            m1 = re.match("^(.*?)(\d+ \{} \d+)(.*)$".format(pref), s)
            s = "{}{}{}".format(m1.groups(1)[0], eval(m1.groups(1)[1]), m1.groups(1)[2])

    while s.count('+') + s.count('*') > 0:
        m1 = re.match("(\d+ [\+\*]{1} \d+)(.*)$", s)
        s = "{}{}".format(eval(m1.groups(1)[0]), m1.groups(1)[1])

    return s

###############################################################################

lines = get_string_input('prob35.txt')
sum35, sum36 = 0, 0
for line in lines:
    sum35 += int(evaluate(line))
    sum36 += int(evaluate(line, '+'))
logging.info("Problem 35: The sum of the evaluated lines is {}.".format(sum35,))
logging.info("Problem 36: The sum of the evaluated lines is {}.".format(sum36,))


2020-12-27 09:28:29 INFO: Got 374 lines from prob35.txt.
2020-12-27 09:28:29 INFO: Problem 35: The sum of the evaluated lines is 12956356593940.
2020-12-27 09:28:29 INFO: Problem 36: The sum of the evaluated lines is 94240043727614.


## Day 19: Monster Messages

### Problem 37:  How many messages completely match rule 0?

### Problem 38:  After updating rules 8 and 11, how many messages completely match rule 0?

In [21]:
def get_string_input(filename):
    retval = None
    with open(filename) as IN:
        retval = IN.read().splitlines()
    logging.info("Got {} lines from {}.".format(len(retval),filename))
    return retval


def get_syntax_and_messages(lines):
    ''' Parse input and return two structures: syntax and messages
            syntax[rule] = [ rule_list1, rule_list2 ]
             - eg input line = "7: 99 32 | 24 86" --> syntax[7] = [[99,32], [24,86]]
            message is a list of strings
    '''
    rule_re = re.compile('^(\d+): (.*)$')
    state = 'syntax'
    syntax = dict()
    for i, line in enumerate(lines):
        m1 = rule_re.match(line)
        if state == 'syntax' and m1 == None:
            state = 'messages'
        elif state == 'syntax':
            key, subrules = m1.groups(1)
            if subrules == '"a"':
                syntax[key] = [['a']]  # easier to keep this as a list
            elif subrules == '"b"':
                syntax[key] = [['b']]
            else:
                syntax[key] = [s.split(' ') for s in subrules.split(' | ')]
        elif state == 'messages':
            messages = lines[i:]
            break
    return syntax, messages


def get_matched_for_list(message, rule_list, syntax):
    '''  Return a set of valid message[:k] that match the given rule_list
            message   = a string;  e.g.  'ababaaab'
            rule_list = a list of integers;  e.g. [8, 11] or [4]
            syntax    = dict containing rules;  syntax[rule_num] = [ rule_list1, rule_list2 ]
    '''
    retval = set()
    # get matches for first rule
    prefixes = get_matched_for_rule(message, rule_list[0], syntax)
    if len(rule_list) == 1:
        # if list contains 1 rule, return the matches found
        retval = prefixes
    else:
        # list contains more rules.
        # For each prefix already found, remove it from message and try to match rest
        #   of rule_list to the remaining message.  Then add prefix to each suffix and
        #   add resulting string to retval as a match for the rule_list
        for p in prefixes:
            suffixes = get_matched_for_list(message[len(p):], rule_list[1:], syntax)
            for s in suffixes:
                retval.add(p + s)
    return retval


def get_matched_for_rule(message, rule, syntax):
    '''  Return a set of valid message[:k] that match the given rule
            message   = a string;  e.g.  'ababaaab'
            rule_list = a list of integers;  e.g. [8, 11] or [4]
            syntax    = dict containing rules;  syntax[rule_num] = [ rule_list1, rule_list2 ]
    '''
    retval = set()
    if len(message) == 0:
        pass
    elif rule in ["a", "b"]:
        # If rule is a string char that matches first char of message, return it.
        if message[0] == rule:
            retval.add(rule)
        # else chars don't match, return empty set
    else:
        # rule is a number, attempt to match message to each rule_list in syntax[rule]
        for rule_list in syntax[rule]:
            retval = retval.union(get_matched_for_list(message, rule_list, syntax))
    return retval

#######################################################################################

lines = get_string_input('prob37.txt')
syntax, messages = get_syntax_and_messages(lines)

match_count = 0
match_count = sum([m in get_matched_for_rule(m, '0', syntax) for m in messages])
logging.info("Problem 37: The number of messages that match rule 0 is {}.".format(match_count,))

# Change rules per problem description for Problem 38
syntax['8'] = [['42'], ['42','8']]
syntax['11'] = [['42', '31'], ['42','11','31']]
match_count = sum([m in get_matched_for_rule(m, '0', syntax) for m in messages])
logging.info("Problem 38: The number of messages that match rule 0 is {}.".format(match_count,))


2020-12-27 09:28:29 INFO: Got 631 lines from prob37.txt.
2020-12-27 09:28:30 INFO: Problem 37: The number of messages that match rule 0 is 220.
2020-12-27 09:28:32 INFO: Problem 38: The number of messages that match rule 0 is 439.


## Day 20: Jurassic Jigsaw

### Problem 39: What do you get if you multiply together the IDs of the four corner tiles?

### Problem 40: How many # are not part of a sea monster?

In [22]:
class Tile:
    def __init__(self, tileId, face):
        self.tileId = tileId  #integer
        self.face = face      #face[x][y] in ['.', '#']
        self.d = len(face[0])
        self.edgeNums = []
        self.orientation = 0  # index of [e, r, r^2, r^3, sr, sr^2, sr^3, s]
        self.set_tile_values()
        # mult on right by sr^3  moves top to left
        # so if g = left_index on top, then g*sr^3 = left_index on left
        self.left_top_transposition = [6, 5, 4, 7, 2, 1, 0, 3]
        # if top at x, left_side at left_top_transposition[x]
        # if left at x, top_side at left_top_transposition[x]

    ############################################################################

    def rotate(self):
        ''' Rotates piece by 90 degrees (clockwise) '''
        new_face = []
        for col in range(self.d):
            row = ''.join([self.face[r][col] for r in np.arange(self.d-1, -1, -1)])
            new_face.append(row)
        self.face = new_face
        self.orientation += 1
        if self.orientation % 4 == 0:
            self.orientation -= 4

    ############################################################################
            
    def flip(self):
        '''This function works a lot like the rotate function'''
        self.face = [''.join(reversed(self.face[r])) for r in range(self.d)]
        self.orientation = 7 - self.orientation

    ############################################################################

    def output(self):
        logging.info("\n     {}\n".format('\n     '.join(self.face),))                     

    ############################################################################

    def get_top_row_val(self):
        return sum([2**(9-i) for i, val in enumerate(self.face[0]) if val == '#'])

    ############################################################################

    def get_bottom_row_val(self):
        return self.edgeNums[ [5, 4, 7, 6, 1, 0, 3, 2][self.orientation] ]
    
    ############################################################################

    def get_left_side_val(self):
        return self.edgeNums[ [6, 5, 4, 7, 2, 1, 0, 3][ self.orientation ] ]
    
    ############################################################################

    def get_right_side_val(self):
        return self.edgeNums[ [3, 0, 1, 2, 7, 4, 5, 6][self.orientation] ]
    
    ############################################################################

    def set_tile_values(self):
        # Get values for [e, r, r^2, r^3, sr, sr^2, sr^3, s]
        for _ in range(8):
            self.edgeNums.append(self.get_top_row_val())
            self.step()

    ############################################################################

    def step(self):
        if (self.orientation + 1) % 4 == 0:
            self.flip()
        else:
            self.rotate()
                
    ############################################################################

    def fit_tile(self, left, top, all_sides):
        retval = False
        #Try to flip and rotate tile to get left and top sides to match left and top
        if left in self.edgeNums and top in self.edgeNums:
            while self.orientation != self.edgeNums.index(top):
                self.step()
            if self.get_left_side_val() == left:
                retval = True

        elif top in self.edgeNums and left == None:
            while self.orientation != self.edgeNums.index(top):
                self.step()
            if all_sides[ self.get_left_side_val() ] == 1:
                retval = True

        elif left in self.edgeNums and top == None:
            left_val_index = self.edgeNums.index(left)  # index of left value
            top_index = self.left_top_transposition[ left_val_index ]
            while self.orientation != top_index:
                self.step()
            if all_sides[ self.get_top_row_val() ] == 1:
                retval = True

        elif left == None and top == None:
            for _ in range(8):
                top_val_count = all_sides[ self.get_top_row_val() ]
                left_side_count = all_sides[ self.get_left_side_val() ]
                if top_val_count == 1 and left_side_count == 1:
                    retval = True
                    break
                else:
                    self.step()

        return retval
 

    ############################################################################

    def strip_border(self):
        ''' Return tile with edges of face removed. '''
        return Tile(self.tileId, [ self.face[r][1:-1] for r in range(1,self.d-1) ])
                
    ############################################################################

    def is_seamonster(self, row, col):
        ''' Find pattern below beginning where [row, col] is the left-most "O"'''
        #   .#.#...#.###...#.##.O#..
        #   #.O.##.OO#.#.OO.##.OOO##
        #   ..#O.#O#.O##O..O.#O##.##
        retval = False
        if row > 0 and row < self.d - 1 and col < self.d - 19:
            retval = True
            for x, y in [[0,0],[1,-1],[4,-1],[5,0],[6,0],[7,-1],[10,-1],[11,0],
                         [12,0],[13,-1],[16,-1],[17,0],[18,0],[18,1],[19,0]]:
                if self.face[row - y][col + x] != '#':
                    retval = False
                    break
        return retval
        
    ############################################################################

    def find_seamonsters(self):
        ''' Return count of seamonsters found in every orientation of tile. '''
        retval = 0
        for step_count in range(8):
            for row in range(self.d):
                for col in range(self.d):
                    if self.is_seamonster(row, col) == True:
                        retval += 1
            self.step()
        return retval

    ############################################################################

    def count_hashtags(self):
        return sum([self.face[r].count('#') for r in range(self.d)])
    
    ############################################################################

####################################################################################
        
def process_tiles(lines):
    retval = dict()
    state = None
    for line in lines:
        m1 = re.match('Tile (\d+):$', line)
        if m1 != None:
            tileId = int(m1.groups(1)[0])
            state = 'tile'
            tile = [] 
        elif re.match('^\s*$', line) != None:
            retval[tileId] = Tile(tileId, tile) 
            state = None
            tile = [] 
        elif state == 'tile':
            tile.append(line)

    if len(tile) > 0:
        retval[tileId] = Tile(tileId, tile) 

    return retval


def count_edges(tiles):
    ''' Return a dictionary containing the counts among all tiles of occurrences
            of edgeNums.
        retval[edgeNum] = number of edges having edgeNum    
    '''
    retval = dict()
    for tid in tiles:
        for s in tiles[tid].edgeNums:
            if retval.get(s, None) == None:
                retval[s] = 0
            retval[s] += 1
    return retval

    
def assemble_grid(tiles):
    ''' Return list of lists with each sublist a list of Tile() objects.
         The tiles are assembled in retval in rows, each row from left to right.
         For a tile to be added, the edges must match on the left and above.
         Tiles on the top row must have an edge that is unique among tiles.
         Tiles on the left edge must have an edge that is unique among tiles.
         
         This construction assumes a solution exists and that pieces only "fit"
         in one way... i.e. no recursion is necessary... i.e.  greedy construction.
    '''
    retval = []
    # all_sides[edgeNum] = number of times edgeNum observed among all tiles
    all_sides = count_edges(tiles)
    
    # assuming tiles arranged in square grid
    grid_dim = int(math.sqrt(len(tiles)))
    used = set()  # Store used tiles.

    for row in range(grid_dim):
        retval.append([])
        for col in range(grid_dim):
            top  = None if (row == 0) else retval[row-1][col].get_bottom_row_val()
            left = None if (col == 0) else retval[row][col-1].get_right_side_val()
            fit = False
            for tid in tiles:
                if tid not in used and tiles[tid].fit_tile(left, top, all_sides) == True:
                    retval[row].append(tiles[tid])
                    used.add(tid)
                    fit = True
                    break
            if fit == False:
                logging.error("No fit")
                raise

    return retval

##########################################################################################

lines = get_string_input('prob39.txt')

tiles = process_tiles(lines)  # tiles[tileId] = Tile()
grid = assemble_grid(tiles)   # grid[row][col] = Tile()

prob39 = grid[0][0].tileId * grid[0][-1].tileId * grid[-1][0].tileId * grid[-1][-1].tileId
logging.info("Problem 39: The product of the four corners is {}.".format(prob39,))

large_face = [ '' ] * (len(grid) * (grid[0][0].d - 2))
for row in range(len(grid)):
    for col in range(len(grid)):
        new_tile = grid[row][col].strip_border()
        # add rows of new_tile to larger face
        for r in range(new_tile.d):
            large_face[row * new_tile.d + r] += new_tile.face[r]

large_tile = Tile(777, large_face)
prob40 = large_tile.count_hashtags() - (15*large_tile.find_seamonsters())
logging.info("Problem 40: The number of '#' not part of seamonsters is {}.".format(prob40,))


2020-12-27 09:28:32 INFO: Got 1727 lines from prob39.txt.
2020-12-27 09:28:32 INFO: Problem 39: The product of the four corners is 17032646100079.
2020-12-27 09:28:32 INFO: Problem 40: The number of '#' not part of seamonsters is 2006.


## Day 21: Allergen Assessment

### Problem 41: How many times do any of the safe ingredients appear?

### Problem 42: What is your canonical dangerous ingredient list?

In [23]:
def process_allergens(lines):
    retval = []
    for line in lines:
        m1 = re.match('(.*) \(contains (.*)\)', line)
        if m1 == None:
            logging.error("Could not parse line.")
        else:
            retval.append([set(m1.groups(1)[0].split(' ')), 
                           set(m1.groups(1)[1].split(', '))])
    return retval

#############################################################################

#lines = get_string_input('prob41test.txt')
lines = get_string_input('prob41.txt')
data = process_allergens(lines)  # data = [[stuffs, ]]

# Get solution, sol[allg] = set(possible stuff)
sol = dict()
for stuffs, allgs in data:
    for allg in allgs:
        if sol.get(allg, None) == None:
            sol[allg] = stuffs
        else:
            sol[allg] = sol[allg].intersection(stuffs)


# Iteratively determine which stuff contains which allg
done = False
while not done:
    done = True
    for allg in sorted(sol.keys(), key = lambda x: len(sol[x])):
        if len(sol[allg]) != 1:
            done = False   # Iterate until all allgs have one stuff
        else:   # stuff contains allg
            # Remove stuff from other allg sets.
            stuff = list(sol[allg])[0] 
            for a in sol.keys():
                if a != allg:
                    sol[a].discard(stuff)


allStuffs = set.union(*[stuffs for stuffs, _ in data])
unsafeStuffs = set.union(*[sol[allg] for allg in sol.keys() if len(sol[allg]) == 1])
safeStuffs = allStuffs.difference(unsafeStuffs)
safeCount = sum([len(safeStuffs.intersection(stuffs)) for stuffs, _ in data])
logging.info("Problem 41: Safe Ingredients listed {} times.".format(safeCount,))
        
unsafeList = ','.join([list(sol[x])[0] for x in sorted(sol.keys())])
logging.info("Problem 42: Dangerous ingredients list: {}".format(unsafeList,))

2020-12-27 09:28:32 INFO: Got 30 lines from prob41.txt.
2020-12-27 09:28:32 INFO: Problem 41: Safe Ingredients listed 1679 times.
2020-12-27 09:28:32 INFO: Problem 42: Dangerous ingredients list: lmxt,rggkbpj,mxf,gpxmf,nmtzlj,dlkxsxg,fvqg,dxzq


## Day 22: Crab Combat

### Problem 43:  In Combat, what is the winning player's score?

### Problem 44:  In Recursive Combat, what is the winning player's score?

In [24]:
def process_cards(lines):
    player = -1
    decks = [ [], [] ]
    for line in lines:
        m1 = re.match('^Player .*$', line)
        if m1 != None:
            player += 1
        elif re.match('^$', line) == None:
            decks[player].append(int(line))
    return decks

def get_deck_state(d0, d1):
    return "{}|{}".format(','.join([str(x) for x in d0]), ','.join([str(x) for x in d1]))

def war(d0, d1, recursive = False):
    states = set()
    winner = None
    while winner == None:
        state = get_deck_state(d0, d1)
        if recursive and state in states:
            winner = 0
        elif len(d0) == 0:
            winner = 1
        elif len(d1) == 0:
            winner = 0
        else:
            states.add(state)
            mine, crab = d0[0], d1[0]
            d0, d1 = d0[1:], d1[1:]

            if recursive and mine <= len(d0) and crab <= len(d1):
                round_winner, _, _ = war(d0[:mine], d1[:crab], recursive)
            else:
                round_winner = 0 if mine > crab else 1
            
            d0.extend([mine, crab]) if round_winner == 0 else d1.extend([crab, mine]) 
        # break if winner != None
    return winner, d0, d1

#############################################################################

lines = get_string_input('prob43.txt')

decks = process_cards(lines)
winner, decks[0], decks[1] = war(decks[0], decks[1], False)
score = sum([(len(decks[winner]) - i) * val for i, val in enumerate(decks[winner]) ])
logging.info("Problem 43: Player {} wins Combat with a score of {}.".format(winner+1, score))

decks = process_cards(lines)
winner, decks[0], decks[1] = war(decks[0], decks[1], True)
score = sum([(len(decks[winner]) - i) * val for i, val in enumerate(decks[winner]) ])
logging.info("Problem 44: Player {} wins Recursive Combat with a score of {}.".format(winner+1, score))



2020-12-27 09:28:32 INFO: Got 53 lines from prob43.txt.
2020-12-27 09:28:32 INFO: Problem 43: Player 1 wins Combat with a score of 31308.
2020-12-27 09:28:50 INFO: Problem 44: Player 1 wins Recursive Combat with a score of 33647.


## Day 23:  Crab Cups

### Problem 45:  After 100 moves, what order are the cups in?

### Problem 46: After 10000000 moves with 1000000 cups, what is the product of the two cups following cup 1?

In [25]:
cup_str = '284573961'
#cup_str_test = '389125467'

def crab_shuffle(cups):
    active, retval = cups[0], cups[4:]
    dest = active
    while dest not in retval:
        dest = dest - 1 if dest > 1 else len(cups)
    dest_index = retval.index(dest)
    retval = retval[:dest_index+1] + cups[1:4] + retval[dest_index+1:] + [active]
    return retval

def rotate_solution(cups):
    one_index = cups.index(1)
    return cups[one_index + 1:] + cups[:one_index] + [1]

cups = [int(x) for x in cup_str]
moves = 100
for step in range(moves):
    cups = crab_shuffle(cups)
solution = ''.join([str(x) for x in rotate_solution(cups)[:-1]])
logging.info("Problem 45. After {} steps, the sequence is {}".format(step+1, solution))

################################################################

class Node:
    def __init__(self, val):
        self.val = val
        self.next = None
        
def crab_shuffle2(root, numCups, allCups):
    movers = root.next
    new_root = movers.next.next.next
    root.next = new_root

    dest_val = root.val - 1 if root.val > 1 else numCups
    while dest_val in [movers.val, movers.next.val, movers.next.next.val]:
        dest_val = dest_val - 1 if dest_val > 1 else numCups

    movers.next.next.next = allcups[dest_val].next
    allcups[dest_val].next = movers
    return new_root

def rotate_solution2(root, allcups):
    return allcups[1].next
 

numCups = 1000000
moves = 10000000

cups = [int(x) for x in cup_str] + [x for x in range(10,numCups+1)]
allcups = dict()
root = Node(cups[0])
last = root
allcups[cups[0]] = last
for i in range(1,numCups):
    last.next = Node(cups[i])
    last = last.next
    allcups[cups[i]] = last
last.next = root

for step in range(moves):
    root = crab_shuffle2(root, numCups, allcups)
root = rotate_solution2(root, allcups)
logging.info("Problem 46: Product of {} and {} is {}".format(root.val, root.next.val, root.val*root.next.val))


2020-12-27 09:28:50 INFO: Problem 45. After 100 steps, the sequence is 26354798
2020-12-27 09:29:14 INFO: Problem 46: Product of 367311 and 452745 is 166298218695


## Day 24:  Lobby Layout

### Problem 47: After all of the instructions have been followed, how many tiles are left with the black side up?

### Problem 48: How many tiles will be black after 100 days?

In [26]:
def get_alldirs(lines):
    retval = []
    directions_re = re.compile("^(se|sw|nw|ne|w|e)(.*)$")
    for line in lines:
        dirs = []
        m1 = directions_re.match(line)
        while m1 != None:
            dirs.append(m1.groups(1)[0])
            m1 = directions_re.match(m1.groups(1)[1])
        retval.append(dirs)
    return retval

def follow_dirs(pos, dirs, moves):
    retval = pos[:]
    for d in dirs:
        retval[0] += moves[d][0]
        retval[1] += moves[d][1]
    return retval

def get_black_tiles(dirs, moves):
    retval = set()  # retval[(x,y)] = 'b' # hold black tiles
    for dirs in alldirs:
        loc = tuple(follow_dirs([0,0], dirs, moves))
        if loc in retval:
            retval.remove(loc)
        else:
            retval.add(loc)
    return retval    

def add_vecs(v1, v2):
    return v1[0] + v2[0], v1[1] + v2[1]

def get_adj_black(pos, tiles, moves):
    return sum([add_vecs(pos, m) in tiles for m in moves.values()])

################################################################

# Hex Grid with 2-dimensions
# 2 1 0 1 2 3
#  2 1 0 1 2 3
#   2 1 * 1 2 3
#    2 1 0 1 2 3
#     2 1 0 1 2 3   
#      2 1 0 1 2 3

lines = get_string_input('prob47.txt')
alldirs = get_alldirs(lines)
moves = {'e':[1,0], 'w':[-1,0], 'ne':[1,1], 
         'nw':[0,1], 'se':[0,-1], 'sw':[-1,-1]}
tiles = get_black_tiles(alldirs, moves)
logging.info("Problem 47: The number of black tiles is {}.".format(len(tiles),))
        
rounds = 100
for i in range(rounds):
    new_tiles = set()
    xs, ys = [v[0] for v in tiles], [v[1] for v in tiles]
    minx, maxx, miny, maxy = min(xs), max(xs), min(ys), max(ys)

    for x, y in tt.product(range(minx-1, maxx+2), range(miny-1, maxy+2)):
        pos = tuple([x,y])
        num_adj_black = get_adj_black(pos, tiles, moves)
        if pos not in tiles and num_adj_black == 2:
            new_tiles.add(pos)
        elif pos in tiles and num_adj_black in [1,2]:
            new_tiles.add(pos)
    tiles = new_tiles
logging.info("Problem 48: After round {} the number of black tiles is {}.".format(i + 1, len(tiles)))


2020-12-27 09:29:14 INFO: Got 466 lines from prob47.txt.
2020-12-27 09:29:14 INFO: Problem 47: The number of black tiles is 400.
2020-12-27 09:29:16 INFO: Problem 48: After round 100 the number of black tiles is 3768.


## Day 25: Combo Breaker

### Problem 49: What encryption key is the handshake trying to establish?

### Problem 50: Click a button.

In [27]:
def get_sec(base, modulus, pub):
    retval = 1
    sub_val = base
    while sub_val != pub:
        retval += 1    
        sub_val = (sub_val*base) % modulus
    return retval

def get_enc_val(base, modulus, secret):
    retval = 1
    for _ in range(secret):
        retval = (retval * base) % modulus
    return retval

base, modulus = 7, 20201227
#card_pub, door_pub = 5764801, 17807724
card_pub, door_pub = 15628416, 11161639

#card_sec = get_sec(base, modulus, card_pub)
door_sec = get_sec(base, modulus, door_pub)
enc_val = get_enc_val(card_pub, modulus, door_sec)
#enc_val = get_enc_val(door_pub, modulus, card_sec)
logging.info("Problem 49: The encryption key is {}.".format(enc_val,))


2020-12-27 09:29:20 INFO: Problem 49: The encryption key is 19774660.
